This notebook provides an example workflow for caking xray diffraction patterns using pyFAI.

In [ ]:
import pathlib
import logging
import math
import warnings

import pyFAI.azimuthalIntegrator
from pyFAI.gui import jupyter
import fabio
import numpy as np

# Visualisation of patterns
## Calibration image
The calibration image can be opened and read into an array which can be directly plotted.

In [ ]:
image = fabio.open('calibration/DLS_CeO2_1200mm.tif')
print("image:", image)

calibration_image_array = image.data
print("calibration_image_array:", type(calibration_image_array), calibration_image_array.shape, calibration_image_array.dtype)

jupyter.display(calibration_image_array)

## Pattern image
The pattern file can be opened in the same way.

In [ ]:
image = fabio.open("data/pixium_03150.tif")

pattern_image_array = image.data
print("pattern_image_array:", type(pattern_image_array), pattern_image_array.shape, pattern_image_array.dtype)

jupyter.display(pattern_image_array)

Because the data is in NumPy arrays, we can do things like subtract the image. This could be useful for subtracting a dark image of the detector, for example.

In [ ]:
example_subtraction = pattern_image_array - calibration_image_array
jupyter.display(example_subtraction)

# Example workflows for caking
First, we need to load the calibration, which contains information about our beamline setup like the orientation
of the detector relative to the image. The calibration is stored in an `azimuthal integrator`.

*Notes on creating a .poni calibration file using [Dioptas](http://www.clemensprescher.com/programs/dioptas) are found on the [LightForm Wiki](https://lightform-group.github.io/wiki/tutorials/sxrd-caking-dioptas)*

In [ ]:
ai = pyFAI.load("calibration/DLS_CeO2_1200mm.poni")
print("\nIntegrator: \n", ai)

## Azimuthal integration

The `integrate2d` function of the azimuthal integrator object does a 2D azimuthal integration. This
divides the images into sections as a function of azimuthal angle and 2-theta angle.

[TODO] - review all available options and determine those needed for our Diamond and DESY data.

* If provided, the `filename` parameter allows output of the data to a file in 'edf' format.
 This file has the data in ascii format with some metadata. This is not directly the data format that
 xrdfit reads and so we will output our own custom data file later.
* The `method` function allows selection of different methods of averaging. This may be important,
  particularly if you are averaging the data to a low number of bins. More details about the different
  methods can be found in the pyFAI documentation.
* The result is returned as an `Integrate2dResult` which contains NumPy arrays with the caked data.

In [ ]:
result = ai.integrate2d(pattern_image_array,
                        1000,
                        36,
                        azimuth_range=(-180,180),
                        unit="2th_deg",
                        polarization_factor=0.99,
                        method='full_csr',
                        filename = "analysis/integrated.edf")
print('The intensity data is stored in "result.intensity" and has the shape: ', np.shape(result.intensity))
print('The radial bin edges are stored in "result.radial" with shape: ', np.shape(result.radial))
print('The azimuthal bin edges are stored in "result.azimuthal" with shape: ', np.shape(result.azimuthal))

If it is output, we can open the '.edf' file using FabIO.

In [ ]:
cake = fabio.open("analysis/integrated.edf")
print(cake.header)
print("cake:", type(cake.data), cake.data.shape, cake.data.dtype)

## Cake rotation

More details about the orientation of the cakes is explained in the accompanying `pyFAI-sample-rotation` notebook using an example image.

The relation between the cardinal directions perpendicular to the beamline and the azimuthal angle is:

- WEST is -180${^\circ}$ / +180${^\circ}$ 
- SOUTH is -90${^\circ}$
- EAST is 0${^\circ}$ / 360${^\circ}$
- NORTH is +90${^\circ}$

### Comparison to caking in DAWN
pyFAI cakes the data anticlockwise which is the opposite direction to DAWN. pyFAI also starts caking the data from the `West` (-180${^\circ}$) direction (compared to DAWN which starts caking from the East direction).

This means data is caked from -180${^\circ}$ to 0${^\circ}$ to +180${^\circ}$.

And, as shown by the result below, the minimum cake angle in pyFAI is -180${^\circ}$, and so caking will slice along the horizontal, meaning the cake is centred on -175${^\circ}$ cake for 10${^\circ}$ slices, rather than being centred on the horizontal.

In [ ]:
result.azimuthal

For some of the materials we analyse, there is signal in the cardinal directions so it is desirable that the
cakes are centred on the horizontal (and vertical) by applying a rotation to the detector.

In [ ]:
# rotate the detector so that the cardinal direction is in the center of the first cake.
number_of_cakes = 36
first_cake_angle = 360 / number_of_cakes
ai.rot3 = (first_cake_angle / 2) * (math.pi / 180) # convert rotation to radians
print(ai.rot3)

# Caking multiple images

To iterate through multiple images we can create a loop, create an array for the data and then save it as a text file.

xrdfit has a simple data format with the first column being the bin edges of the two-theta angle and the subsequent
columns being the binned intensity data, one column for each cake.

Note that after the azimuthal integration the `np.flip` function is used to reverse the order of the columns of
intensity data. This means that the cakes are ordered clockwise rather than anticlockwise.

In [ ]:
# suppress warnings when TIFFs are read
logging.getLogger("fabio.TiffIO").setLevel(logging.ERROR)

# user inputs
number_of_points = 1000
number_of_cakes = 36

# rotate the detector by half a cake width so that the cardinal direction
# is in the center of the first cake not the edge.
first_cake_angle = 360 / number_of_cakes
ai.rot3 = (first_cake_angle / 2) * (math.pi / 180) # convert rotation to radians

# get a list of the files
image_list = sorted(pathlib.Path("data/").glob("pixium*"))

for image_path in image_list:
    # create empty array
    caked_data = np.zeros((number_of_cakes + 1, number_of_points))
    
    # create an image array and cake the data
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        image = fabio.open(image_path)
    pattern_image_array = image.data
    result2d = ai.integrate2d(pattern_image_array,
                            number_of_points,
                            number_of_cakes,
                            unit="2th_deg",
                            polarization_factor=0.99,
                            method='full_csr')
    
    # flip the intensity data to order cakes clockwise rather than anticlockwise
    intensity = np.flip(result2d.intensity.T, axis=1)
    
    # reshape radial labels to 2D array so they can be attached to the intensity data.
    radial = np.reshape(result2d.radial, (-1, 1))
    
    result_array = np.hstack((radial, intensity))
    
    # write out the caked data to a text file
    np.savetxt(f"analysis/{image_path.stem}.dat", result_array)

This caked dataset is now saved in a format that we can use in [xrdfit](https://xrdfit.readthedocs.io/en/stable/) to
 analyse how the peak profiles change over time.